# This notebook shows how to manage slices

## Configure the environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

## Create slice manager object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host,refresh_token=fabric_refresh_token, project_name='all', scope='all')

## Refresh the ID Token

In [ ]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print("New Refresh Token: {}".format(fabric_refresh_token))
print("Stored new Refresh Token")
%store fabric_refresh_token

## Create Basic Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='Node', site='LBNL')

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=16, disk=100)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='ExampleSlice', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(reservations))
if status == Status.OK:
    print("Reservations created {}".format(reservations))

## Query existing Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

## Retrieve Slice ID and Sliver ID

In [ ]:
#Name of slice to be renewed
TargetSliceName = "ExampleSlice"
TargetNode = "Node"

#Search for Slice ID from Slice Name
slice_id = list(filter(lambda slices: slices.slice_name == TargetSliceName, slices))[0].slice_id

#Search Slice for Target Sliver ID
status, slivers = slice_manager.slivers(slice_id=slice_id)
sliver_id = list(filter(lambda slivers: slivers.name == TargetNode, slivers))[0].reservation_id

## Query Slice Status

In [ ]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))

## Query Sliver Status

In [ ]:
status, sliver = slice_manager.slivers(slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Sliver {}".format(sliver))

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(result))